## Death Penalty Executions

*Source: [Death Penalty Information Center](https://deathpenaltyinfo.org/)*

As part of a class visualization exercise, let's generate some basic data outputs using the DPIC's executions database.

> See the [DPIC factsheet](https://deathpenaltyinfo.org/documents/FactSheet.pdf) for more background, historical figures and past reports.

Data summaries to export:

* Total number of executions
* Total number of executions by year
* Number of executions by state
* Number of executions by state over time
* Executions by race of inmate
* Executions by inmate race, relative to:
  * Total population
  * Incarcerated population


## Read and prepare the data

In [66]:
import re
import pandas as pd
data = pd.read_csv('data/execution_database.csv', parse_dates=['Date'], low_memory=False)
# snake_case column names
data.rename(columns={col: re.sub(r'\s+', '_', col.lower().replace('/', '')) for col in data.columns}, inplace=True)
# Add a year column
data['year'] = data.apply(lambda row: row.date.year, axis=1)
# Filter out malformed records (two records have a lots of NaNs)
data = data[data.year.notnull()]
# Convert year to integer
data.year = data.year.fillna(0).apply(int)

In [196]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1490 entries, 0 to 1491
Data columns (total 15 columns):
date                          1490 non-null datetime64[ns]
name                          1490 non-null object
age                           1490 non-null int64
sex                           1490 non-null object
race                          1490 non-null object
number_race_sex_of_victims    1490 non-null object
state                         1490 non-null object
region                        1490 non-null object
method                        1490 non-null object
juvenile                      1490 non-null object
federal                       1490 non-null object
volunteer                     1490 non-null object
foreign_national              1490 non-null object
county                        1488 non-null object
year                          1490 non-null int64
dtypes: datetime64[ns](1), int64(2), object(12)
memory usage: 186.2+ KB


## Boilerplate dataframes

> Note: there's likely a more idiomatic way to do the following...

We'll need some zeroed out dataframes to simplify the process of setting default counts to zero in downstream `groupby` queries. For example, there were no executions in 1976, 1978 or 1980, and we need these years to have counts of 0 in the final dataframe for visualization purposes. Similary, when we group by state and year, not all states will have executions in a given year.

In [192]:
years_df0 = pd.DataFrame({'year': list(range(1976, 2019))})

In [191]:
states = pd.DataFrame({'state': data['state'].drop_duplicates()})

Perform a cartesian (or cross) join to generate a dataframe containing all combinations of year and state.

> Solution cribbed from this [StackOverflow answer](https://stackoverflow.com/questions/53699012/performant-cartesian-product-cross-join-with-pandas/53699013#53699013)


In [217]:
year_state_df0 = years_df0.assign(key=1)\
    .merge(states.assign(key=1), on='key')\
    .drop('key', 1)\
    .sort_values(['year', 'state'])

## Total executions

In [68]:
total_executions = data.shape[0]
total_executions

1490

### Total By year

Generate a count of executions by year.

In [197]:
yearly = data.groupby(["year"]).size().reset_index().rename(columns={0: 'freq'})

In [225]:
# Join years DF with grouped yearly counts
yearly_counts = pd.merge(years_df0, yearly, on=['year'], how='left')
# Default to zero for missing years and convert "freq" to integer
yearly_counts.freq = yearly_counts.freq.fillna(0).apply(int)
yearly_counts.head()

,year,freq
0,1976,0
1,1977,1
2,1978,0
3,1979,2
4,1980,0


In [201]:
# Export the data for visualization in an external tool
yearly_counts.to_csv('data/executions_yearly_totals.csv', index=False)

## State totals

Generate totals by state.

In [216]:
state_totals = data\
  .groupby(by='state')\
  .size()\
  .reset_index()\
  .rename(columns={0: 'freq'})\
  .sort_values(['freq'], ascending=False)
state_totals.head()

,state,freq
30,TX,558
32,VA,113
24,OK,112
8,FL,97
16,MO,88


In [212]:
state_totals.to_csv('data/executions_state_totals.csv', index=False)

## State totals by year

Generate a state/year totals DataFrame and merge with the pre-initialized DataFrame containing zeroed-out values for all combinations of state and year.

In [220]:
state_year_totals = data\
  .groupby(by=['year', 'state'])\
  .size()\
  .reset_index()\
  .rename(columns={0: 'freq'})\
  .sort_values(['state', 'year'])
state_totals.head()

,state,freq
30,TX,558
32,VA,113
24,OK,112
8,FL,97
16,MO,88


In [227]:
# Join years DF with grouped yearly counts
year_state_all = pd.merge(year_state_df0, state_year_totals, on=['year', 'state'], how='left')
# Default to zero for missing years and convert to count to integer
year_state_all.freq = year_state_all.freq.fillna(0).apply(int)
year_state_all.head()

,year,state,freq
0,1976,AL,0
1,1976,AR,0
2,1976,AZ,0
3,1976,CA,0
4,1976,CO,0


Export a "long version of the data.

In [224]:
year_state_all.to_csv('data/executions_state_yearly_totals.csv', index=False)

Create a wide, or pivoted version of the data, for use in sundry visualization tools.

In [230]:
yearly_state_all_pivoted = year_state_all.pivot(
    index='year',
    columns='state',
    values='freq'
)
yearly_state_all_pivoted.head()

state,AL,AR,AZ,CA,CO,CT,DE,FE,FL,GA,...,OR,PA,SC,SD,TN,TX,UT,VA,WA,WY
year,,,,,,,,,,,,,,,,,,,,,
1976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1979,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [232]:
yearly_state_all_pivoted.to_csv('data/executions_state_yearly_totals_wide.csv')